In [48]:
import pandas as pd

In [49]:
import numpy as np

In [50]:
import csv

In [51]:
import datetime as dt

In [52]:
# Load visualization tools
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [53]:
# Load csv as dataframe
df = pd.read_csv("trump_schedule_leak.csv")

In [54]:
df=df[df['top_category']!='no_data']

In [55]:
# Prepare start/end times for datetime conversion
for col in ['time_start','time_end']:
    df[col]=df['date']+" "+df[col]

In [56]:
# Convert datetime fields to correct datetime format
for col in ['date','time_start','time_end']:
    df[col]=pd.to_datetime(df[col])

In [57]:
# Pull back out time only
for col in ['time_start','time_end']:
    df[col]=df[col].dt.time

In [58]:
# Generate list of all days in range
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

days = [dt.strftime('%Y-%m-%d') for dt in 
       datetime_range(df['date'].min(), df['date'].max(), 
       timedelta(days=1))]

In [59]:
# Generate list of 15-minute chunks
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt.strftime('%H:%M:%S') for dt in 
       datetime_range(datetime(2019, 1, 1, 0), datetime(2019, 1, 2, 0), 
       timedelta(minutes=15))]

In [60]:
# Create list of all categories
cats = ['no_data']+list(df['top_category'].unique())

In [ ]:
cats = cats.sort_values()

In [61]:
# Encode categories as a number
df['top_category_encoding']=[cats.index(cat) for cat in df['top_category']]

In [62]:
# Save numeric encoding information for use later
encodings = {}
for cat in cats:
    encodings[cats.index(cat)]=cat

In [63]:
# Create schedule flag for each 15-minute chunk
for time in dts:
    dttime = dt.datetime.strptime(time, "%H:%M:%S").time()
    df[time]=((df['time_start'] <= dttime) & (dttime < df['time_end']))*1
    df[time]=df[time]*df['top_category_encoding']

In [64]:
schedule = pd.DataFrame(df.filter(items=(['date']+dts))).groupby(by='date').sum()

In [65]:
schedule_encoded = schedule.replace(encodings)

# Import Tweet Data

In [66]:
tweets = pd.read_csv("trump_tweets_id_dt.txt",sep=" ", index_col=0).rename(columns={"DateTime":'datetime',"ID":'id'})

In [67]:
# Convert to datetime
tweets['datetime']=pd.to_datetime(tweets['datetime'])

In [68]:
tweets=tweets.sort_values(by="datetime")

In [69]:
# Localize time zone
tweets['datetime']=pd.DatetimeIndex(pd.to_datetime(tweets['datetime'],unit='ms')).tz_convert('US/Eastern').tz_localize(None)

In [70]:
# Separate out date and time for axes
tweets['date']=tweets['datetime'].dt.date
tweets['time']=tweets['datetime'].dt.time

In [71]:
# Clip tweet dataframe to overlap with schedule data
tweets = tweets[tweets['datetime']>df['date'].min()]

In [72]:
#tweets['time']=[t.strftime('%H:%M') for t in tweets['time']]

In [73]:
tweets.head(5)

,datetime,id,date,time
268,2018-11-07 02:21:51,1060130202418864129,2018-11-07,02:21:51
167,2018-11-07 02:55:35,1060138691778301953,2018-11-07,02:55:35
483,2018-11-07 03:07:51,1060141780878979072,2018-11-07,03:07:51
1206,2018-11-07 03:36:28,1060148982968733696,2018-11-07,03:36:28
718,2018-11-07 03:52:39,1060153052676702208,2018-11-07,03:52:39


In [74]:
tweets=tweets.sort_values(by='time')

In [75]:
tweets=tweets.reset_index()
tweets.drop(columns=['index'], inplace=True)

In [76]:
tweets['time'][1]

datetime.time(0, 52, 4)

In [77]:
tweets['time'].head(10)

0    00:40:50
1    00:52:04
2    00:59:03
3    02:16:08
4    02:18:59
5    02:19:41
6    02:21:51
7    02:25:51
8    02:28:39
9    02:28:50
Name: time, dtype: object

In [78]:
tweets.head(5)

,datetime,id,date,time
0,2018-11-11 00:40:50,1061554334276747264,2018-11-11,00:40:50
1,2018-11-30 00:52:04,1068442531887632384,2018-11-30,00:52:04
2,2018-11-30 00:59:03,1068444287380025344,2018-11-30,00:59:03
3,2018-12-01 02:16:08,1068826073775964160,2018-12-01,02:16:08
4,2018-12-07 02:18:59,1071001119697108992,2018-12-07,02:18:59


# Heatmap (1-Trace)

In [79]:
schedule.columns = [datetime.strptime(t, '%H:%M:%S') for t in schedule.columns]

In [80]:
#schedule.columns = [t.time() for t in schedule.columns]

In [81]:
schedule.columns = [str(t) for t in schedule.columns]

In [82]:
tweets['time'] = [str(t) for t in tweets['time']]

In [85]:
tweets['fakedate']='1900-01-01'

In [91]:
tweets['faketime']=tweets['fakedate']+" "+tweets['time']

In [97]:
tweets['faketime']=pd.to_datetime(tweets['faketime'])

In [113]:
tweets['time'][30]

'02:59:04'

In [114]:
tweets['faketime'][30]

Timestamp('1900-01-01 02:59:04')

In [98]:
tweets['faketime'].dtype

dtype('<M8[ns]')

In [84]:
schedule.columns[1]

'1900-01-01 00:15:00'

In [40]:
schedule.columns.max()

'1900-01-01 23:45:00'

In [378]:
schedule.head(5)

,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,02:00:00,02:15:00,...,21:30:00,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00
date,,,,,,,,,,,,,,,,,,,,,
2018-11-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-11-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-11-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-11-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-11-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
tweets.head(10)

,datetime,id,date,time,fakedate,faketime
0,2018-11-11 00:40:50,1061554334276747264,2018-11-11,00:40:50,1900-01-01,1900-01-01 00:40:50
1,2018-11-30 00:52:04,1068442531887632384,2018-11-30,00:52:04,1900-01-01,1900-01-01 00:52:04
2,2018-11-30 00:59:03,1068444287380025344,2018-11-30,00:59:03,1900-01-01,1900-01-01 00:59:03
3,2018-12-01 02:16:08,1068826073775964160,2018-12-01,02:16:08,1900-01-01,1900-01-01 02:16:08
4,2018-12-07 02:18:59,1071001119697108992,2018-12-07,02:18:59,1900-01-01,1900-01-01 02:18:59
5,2018-11-26 02:19:41,1067015026995879937,2018-11-26,02:19:41,1900-01-01,1900-01-01 02:19:41
6,2018-11-07 02:21:51,1060130202418864129,2018-11-07,02:21:51,1900-01-01,1900-01-01 02:21:51
7,2018-12-20 02:25:51,1075713886874148865,2018-12-20,02:25:51,1900-01-01,1900-01-01 02:25:51
8,2018-12-18 02:28:39,1074989817061093378,2018-12-18,02:28:39,1900-01-01,1900-01-01 02:28:39
9,2018-12-07 02:28:50,1071003598023221248,2018-12-07,02:28:50,1900-01-01,1900-01-01 02:28:50


In [136]:
heatmap = go.Heatmap(
    z=schedule.values,
    x=schedule.columns,
    y=schedule.index,
    text=schedule_encoded.values,
    hoverinfo='text',
    name='Schedule',
    colorscale='Viridis'
)

scatterplot = go.Scatter(
    x = tweets['faketime'],
    y = tweets['date'],
    text = tweets['faketime'],
    hoverinfo='text',
    mode = 'markers',
    name='Tweets',
    marker = dict(
        size = 5,
        color = 'red',
        line = dict(
            color = 'white',
            width = 1,
        )
    )
)

data = [scatterplot,heatmap]
#data = [heatmap]

layout = go.Layout(
    title = "Schedule",
    hovermode='closest',
    xaxis=dict(
        title='Schedule Time Axis',
        range=[schedule.columns[28],schedule.columns.max()]
    ),
    height=600
)

fig = go.Figure(data=data,layout=layout)
py.offline.iplot(fig)
#py.offline.plot(fig,'trump_schedule_tweet_overlay.html')




In [29]:
z = schedule.values

label = schedule_encoded.values
    
hover=label

fig = ff.create_annotated_heatmap(z,colorscale='Portland')
py.offline.iplot(fig, filename='annotated_heatmap')

# Heatmap using multiple traces

In [30]:
# Create an array of data frame schedules; one for each category
dfs = []
for cat in cats:
    # Create a new dataframe for the selected category with date as index and times as columns
    catdf = pd.DataFrame(df[df['top_category']==cat],columns=(['date']+dts))
    catdf = catdf.groupby(by='date').sum()
    dfs.append(catdf)
    # print(catdf.shape)

In [31]:
for cat in cats:
    # Add missing dates with 0's
    for d in df['date'].unique():
        # If the date isn't already listed in this dataframe
        if d not in dfs[cats.index(cat)].index:
            #Add it and fill all columns with zeroes
            dfs[cats.index(cat)].loc[d]=[0] * len(dfs[cats.index(cat)].columns)
        # If the date is already in the dataframe for this category, don't do anything
        else:
            pass
        # IMPORTANT: Replace all zero's with NULL - sets the transparency in the heatmap
        dfs[cats.index(cat)] = dfs[cats.index(cat)].replace(0, np.nan)
    # print(dfs[cats.index(cat)].shape)

In [32]:
# Demo for querying schedule dataframes:
# dfs[cats.index('executive_time')]['08:00']['2018-11-07']

In [33]:
# Set colors for visualization
colors = ['red','orange','yellow','green','blue','purple','brown','black','pink']

In [34]:
cats

['no_data', 'executive_time', 'meeting', 'lunch', 'travel', 'event']

In [35]:
graphs = []

# Thanks etpinard for the trace hack: https://codepen.io/etpinard/pen/GmbVZq?editors=0010

for cat in cats:
    num = cats.index(cat)
    color = colors[num]
    graph = go.Heatmap(
        name=str(cat),
        z=dfs[cats.index(cat)].values,
        x=list(dfs[cats.index(cat)].columns),
        y=list(dfs[cats.index(cat)].index),
        colorscale=[[0, color], [1, color]],
        colorbar=dict(
            lenmode='pixels',
            len=50,
            y=1-(0.1*num),
            tickvals=[0],
            ticktext=[''],
            title=str(cat)
        )
    )
    graphs.append(graph)

In [36]:
layout = go.Layout(
    title = "Trump Schedule",
    hovermode='closest'
)

fig = go.Figure(data=graphs,layout=layout)
py.offline.iplot(fig)